In [33]:
import fasttext
import string
import pandas as pd
import numpy as np

In [17]:
# training our model
model = fasttext.train_unsupervised('data_preprocessed.txt', "cbow", minn = 2, maxn = 13, epoch = 1, dim = 50, thread = 2)

In [24]:
string_ = "Ashu is a bitch's buoy! He doesn't like himself much either."
# this pre_processing will make verse suitable for setnece embeddings.
string_ = string_.lower().translate(str.maketrans('','',string.punctuation)).split()

{'ashu': 1,
 'is': 1,
 'a': 1,
 'bitchs': 1,
 'buoy': 1,
 'he': 1,
 'doesnt': 1,
 'like': 1,
 'himself': 1,
 'much': 1,
 'either': 1}

In [28]:
# a function to get the word frequency of each word in a verse/sentence
# list_of_words: a pre-processed list of words in a verse/sentence.
def word_count(list_of_words):
    # a dictionary keeping frequency for all the unique words
    counts = dict()
    
    for word in list_of_words:
        # if the word is already listed, increasing its count
        if word in counts:
            counts[word] += 1
        # else putting the new word in our dictionary
        else:
            counts[word] = 1

    return counts

In [29]:
# a function to calculate composite sentence/verse embeddings
# verse has to be given in a tokenzied format to this.
def senemb(alpha, verse_tok, model, dim):
    # creating an empty vector
    sentEmb = np.zeros(dim)
    v_count = word_count(verse_tok)
    
    for word in verse_tok:
        prob = alpha / (alpha + v_count[word]/len(verse))
        sentEmb = sentEmb +  model.get_word_vector(word) * prob
        
    return sentEmb/len(verse)

In [51]:
# Here we calculate the sentence embedding vectors for verses

# let's get all the verses and get their vectors
verses = pd.read_csv('verses.csv', usecols=['#verse', 'Translation'])
data = verses['Translation'].to_list()

# a list to store all the vectors
verse_vectors = []

# looping over all the verses
for verse in data:
    # tokenizing and preprocessing the verse
    verse_tok = verse.lower().translate(str.maketrans('','',string.punctuation)).split() 
    verse_vectors.append(senemb(1, verse_tok, model, 50))    

In [52]:
# Here we calculate the sentence embedding vectors for queries

# getting the queries
df = pd.read_csv('queries_syn.csv')
queries = df['Verse'].to_list()

# pre-processing the queries
queries = [query.lower().translate(str.maketrans('','',string.punctuation)).split() for query in queries]

# a list to store all the vectors
query_vectors = []

for query in queries:
    query_vectors.append(senemb(1, query, model, 50))

In [ ]:
# let's find the best matching queries for all our queries
for i, query in enumerate(queries):
    